In [ ]:
import os

try:
    if CWD_UPDATED:
        pass
except:
    os.chdir("..")

CWD_UPDATED = True

print(f"Working directory: {os.getcwd()}")

In [ ]:
from components.rolls.piano import PianoRoll
from components.note import Note
from components.chord import Chord, ChordQuality
from transformers.pitch import Pitch
from transformers.interval import Interval

In [ ]:
key = Pitch("C")

I = Chord(key, ChordQuality.Maj)
IV = Chord(key + Interval("4"), ChordQuality.Maj)
V = Chord(key + Interval("5"), ChordQuality.Maj)

roll = PianoRoll(beats_per_minute=110)

roll.add_stride_pattern(I, roll.Time(0, 0), roll.Time(0, 2))
roll.add_stride_pattern(IV.get_V7(), roll.Time(0, 2), roll.Time(1, 0))
roll.add_stride_pattern(IV, roll.Time(1, 0), roll.Time(1, 2))
roll.add_stride_pattern(V.get_V7(), roll.Time(1, 2), roll.Time(2, 0))
roll.add_stride_pattern(V, roll.Time(2, 0), roll.Time(3, 0))
roll.add_stride_pattern(I, roll.Time(3, 0), roll.Time(3, 2))
roll.add_notes(
    Note(I.root, roll.Time(3, 2), roll.Duration(1)).reoctave_near_pitch(Pitch("C3"))
)

In [ ]:
os.makedirs("playground/output", exist_ok=True)
with open("playground/output/output.mid", "wb") as fout:
    roll.to_midi().writeFile(fout)

In [ ]:
os.system("open -a VLC playground/output/output.mid")